<a href="https://colab.research.google.com/github/Nedu21/Pytorch-deep-learning-projects-/blob/main/Pytorch_practice_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
X = torch.randn(100, 1)

true_w, true_b = 2.5, 0.8

y = true_w * X + true_b + torch.randn(100, 1) * 0.1
Y = true_w * X + true_b

In [3]:
w = torch.randn(1, requires_grad = True)
b = torch.randn(1, requires_grad = True)

In [4]:
learning_rate = 0.01
criterion = nn.MSELoss()
optimizer = torch.optim.SGD([w, b], lr=learning_rate)

In [5]:
epochs = 1000

In [6]:
for epoch in range(epochs):
  ypred = w * X + b
  loss = criterion(ypred, y)
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()

  if (epoch+1) % 200 == 0:
    print(f'---Epoch {epoch+1}---')
    print(f'Loss: {loss.item()}')
    print(f'W: {w.item()}')
    print(f'B: {b.item()}')

---Epoch 200---
Loss: 0.014676672406494617
W: 2.4324748516082764
B: 0.8204909563064575
---Epoch 400---
Loss: 0.01077447459101677
W: 2.490842580795288
B: 0.7952030301094055
---Epoch 600---
Loss: 0.010771030560135841
W: 2.4924497604370117
B: 0.7941873073577881
---Epoch 800---
Loss: 0.010771024972200394
W: 2.4924917221069336
B: 0.7941542267799377
---Epoch 1000---
Loss: 0.010771024972200394
W: 2.4924917221069336
B: 0.7941542267799377


Working with data

In [7]:
# Download the training data from open datasets
train_data = datasets.FashionMNIST(
    # Tells pytorch where to save data. If folder doesn't exist it creates one
    root='data',
    # Specifies you want to get the training split
    train=True,
    # Asks if data is downloaded already if no, download the dataset if yes, skip download
    download=True,
    # Converts image to pytorch tensor and scales pixel value from [0, 255] to  [0, 1]
    transform=transforms.ToTensor()
)

# Download the test data from open datasets
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

In [8]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [9]:
# To accelerate operations in the neural network, we move it to the accelerator such as CUDA, MPS, MTIA, or XPU
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [10]:
class NeuralNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNet().to(device)
print(model)

NeuralNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [12]:
print(len(train_dataloader.dataset))
print(len(train_dataloader))

60000
938


In [13]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    yhat = model(X)
    loss = loss_fn(yhat, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X) # ie for batch 100 current = (100 + 1) * 64
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [14]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()

  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      yhat = model(X)
      test_loss += loss_fn(yhat, y).item()
      correct += (yhat.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
epochs = 5
for epoch in range(epochs):
  print(f'Epoch {epoch+1}\n-------------------------------')
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295148  [   64/60000]
loss: 2.291176  [ 6464/60000]
loss: 2.263776  [12864/60000]
loss: 2.245845  [19264/60000]
loss: 2.242278  [25664/60000]
loss: 2.240097  [32064/60000]
loss: 2.228317  [38464/60000]
loss: 2.203719  [44864/60000]
loss: 2.194413  [51264/60000]
loss: 2.188041  [57664/60000]
Test Error: 
 Accuracy: 40.0%, Avg loss: 2.162293 

Epoch 2
-------------------------------
loss: 2.161995  [   64/60000]
loss: 2.133652  [ 6464/60000]
loss: 2.138717  [12864/60000]
loss: 2.116067  [19264/60000]
loss: 2.055939  [25664/60000]
loss: 2.004242  [32064/60000]
loss: 1.993893  [38464/60000]
loss: 1.957140  [44864/60000]
loss: 1.967715  [51264/60000]
loss: 1.938533  [57664/60000]
Test Error: 
 Accuracy: 53.6%, Avg loss: 1.903391 

Epoch 3
-------------------------------
loss: 1.918690  [   64/60000]
loss: 1.887627  [ 6464/60000]
loss: 1.839426  [12864/60000]
loss: 1.852283  [19264/60000]
loss: 1.739183  [25664/60000]
loss: 1.712665  [32064/600

Saving models

In [17]:
torch.save(model.state_dict(), 'model.pth')
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Loading models

In [18]:
model = NeuralNet().to(device)
model.load_state_dict(torch.load('model.pth', weights_only=True))

<All keys matched successfully>

Making predictions

In [21]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[1][0], test_data[1][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Pullover", Actual: "Pullover"
